In [23]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
metric = 'cosine'
user = 1
n = 10

In [3]:
data = pd.read_json('../data/ratings_data.json', orient = 'split')
n_users = data['userId'].unique().shape[0]
n_items = data['movieId'].unique().shape[0]
users = data['userId'].unique()
movies = data['movieId'].unique()

In [4]:
train_data, test_data = train_test_split(data, test_size=0.25)

In [5]:
train_data_matrix = pd.DataFrame(np.zeros((n_users, n_items)), columns = movies, index = users)
for line in train_data.itertuples():
    train_data_matrix.at[line[4], line[1]] = line[2]

In [6]:
test_data_matrix = pd.DataFrame(np.zeros((n_users, n_items)), columns = movies, index = users)
for line in test_data.itertuples():
    test_data_matrix.at[line[4], line[1]] = line[2]

In [7]:
item_similarity = 1 - pairwise_distances(train_data_matrix.transpose(), metric=metric)

In [8]:
def predict(ratings, similarity):
    pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [9]:
item_prediction = pd.DataFrame(predict(train_data_matrix.values, item_similarity), columns = movies, index = users)

In [14]:
def recommend(ratings, user, n = 10):
    user_movies = pd.DataFrame(train_data_matrix.loc[user])
    user_movies.columns = ['rating']
    viewed_movies = user_movies[user_movies['rating'] > 0].index
    predicted_ratings = pd.DataFrame(ratings.loc[user])
    predicted_ratings.columns = ['rating']
    recommended_movies = predicted_ratings.drop(viewed_movies).sort_values(['rating'], ascending=[0]).head(n).index.values

    return recommended_movies

In [24]:
def evaluate(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()

    return sqrt(mean_squared_error(prediction, ground_truth))

In [26]:
error = evaluate(item_prediction.values, test_data_matrix.values)

In [27]:
print(f'Error: {error}')

Error: 3.4463985756299436
